<a href="https://colab.research.google.com/github/RummanAli/FYP2022-KnowledgeVision/blob/main/G1020_KFoldCrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-fold-Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/g1020-polygons/G1020.csv')
partition = {'train': [],'validation' : []}
for i in range(900):
  partition['train'].append(df['imageID'][i])
for i in range(900,1020):
  partition['validation'].append(df['imageID'][i])
labels = dict((val, out ) for val,out in zip(df['imageID'],df['binaryLabels']))

In [ ]:
labels2 = dict((val, out ) for val,out in zip(df['patientID'],df['binaryLabels']))

In [ ]:
patient_labels = []
for x in df['patientID'].unique():
  patient_labels.append(labels2[x])

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
skf = StratifiedKFold(n_splits=6, random_state=42, shuffle=True)
train_folds = []
val_folds = []
for train_index, val_index in skf.split(df['patientID'].unique(), patient_labels):
  train_folds.append(train_index)
  val_folds.append(val_index)
  

In [ ]:
folds = []
for train_fold,val_fold in zip(train_folds,val_folds):
  fold = {'train': [],'validation' : []}
  for x in train_fold:
    a = df['patientID'].unique()[x]
    index = df.index[df['patientID']== a].tolist()
    for img in index:
      fold['train'].append(df['imageID'][img])

  for y in val_fold:
    a = df['patientID'].unique()[y]
    index = df.index[df['patientID']== a].tolist()
    for img in index:
      fold['validation'].append(df['imageID'][img])
  folds.append(fold)

# Model by Author

In [ ]:
from torchvision.models import Inception3
from fastai.vision import *
import os
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import matplotlib
from sklearn.metrics import classification_report

In [ ]:
def load_model():
  v3 = Inception3(2,init_weights=True)
  state_dict = torch.load('/content/drive/MyDrive/bestmodel.pth')
  v3.load_state_dict(state_dict['model'])
  return v3


def get_outputs(v3,fold):
  data_path = '/content/drive/MyDrive/g1020-polygons'
  outputs = []
  for images in fold['validation']:
    image_path = os.path.join(data_path,images[:-4]+'.png')
    img = Image.open(image_path)
    img.load()
    background = Image.new("RGB", img.size, (255, 255, 255))
    background.paste(img, mask = img.split()[3])
    img = background.resize((299, 299), Image.ANTIALIAS)
    img = np.expand_dims(np.array(img),axis = 0)
    img = np.reshape(img,(1,3,299,299))
    v3.eval()
    img = torch.tensor(img)
    out = torch.argmax(torch.exp(v3(img.float())))
    outputs.append(out)
  return outputs

In [ ]:
for i in range(6):
  outputs = get_outputs(load_model(),folds[i])
  y_true = []
  for x in folds[i]['validation']:
    y_true.append(labels[str(x)])
  print(classification_report(y_true,outputs))

              precision    recall  f1-score   support

           0       0.75      0.17      0.27       127
           1       0.27      0.85      0.41        47

    accuracy                           0.35       174
   macro avg       0.51      0.51      0.34       174
weighted avg       0.62      0.35      0.31       174

              precision    recall  f1-score   support

           0       0.82      0.25      0.38       132
           1       0.23      0.81      0.35        36

    accuracy                           0.37       168
   macro avg       0.53      0.53      0.37       168
weighted avg       0.70      0.37      0.38       168

              precision    recall  f1-score   support

           0       0.67      0.25      0.36       122
           1       0.33      0.75      0.46        61

    accuracy                           0.42       183
   macro avg       0.50      0.50      0.41       183
weighted avg       0.56      0.42      0.39       183

              preci

In [ ]:
print(classification_report(y_true,outputs))

              precision    recall  f1-score   support

           0       0.75      0.17      0.27       127
           1       0.27      0.85      0.41        47

    accuracy                           0.35       174
   macro avg       0.51      0.51      0.34       174
weighted avg       0.62      0.35      0.31       174



# Pre-Trained Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Add, Reshape,Conv2D,MaxPooling2D,ZeroPadding2D,experimental,AveragePooling2D,Concatenate,Dropout,Activation
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow
from PIL import Image

In [ ]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=1, dim=(299,299), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = Image.open('/content/drive/MyDrive/g1020-polygons/' + ID[:-4] + '.png')
            background = Image.new("RGB", img.size, (255, 255, 255))
            background.paste(img, mask = img.split()[3])
            X[i,] = np.array(background.resize((299, 299), Image.ANTIALIAS))
            # Store class
            y[i] = self.labels[ID]

        return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
def pre_trained_model():
    input = Input((299,299,3))
    c1l1  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(input)
    c1l2  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l1)
    ces2  = Add()([c1l2, c1l1])
    c1l3  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l2)
    ces3  = Add()([c1l3, ces2])
    ces3  = Flatten()(c1l3)
    ces3  = Dense(2)(ces3)
    ces3  = Activation('softmax')(ces3)
    return Model(inputs=input, outputs=ces3)

In [ ]:
for i in range(6):
  #y_true_train = []
  #for x in folds[i]['train']:
  #  y_true_train.append(labels[str(x)])
  #y_true_test = []
  #for x in folds[i]['validation']:
  #  y_true_test.append(labels[str(x)])
  training_generator = DataGenerator(folds[i]['train'], labels)
  validation_generator = DataGenerator(folds[i]['validation'], labels)
  opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
  pretrained_model = pre_trained_model()
  pretrained_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
  history = pretrained_model.fit_generator(training_generator,validation_data=validation_generator,epochs = 20)
  tf.keras.models.save_model(pretrained_model,'drive/MyDrive/pretrained_fold'+str(i))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


Epoch 1/20
846/846 [==============================] - 309s 364ms/step - loss: 336.3771 - accuracy: 0.6584 - val_loss: 0.5906 - val_accuracy: 0.7241
Epoch 2/20
846/846 [==============================] - 37s 43ms/step - loss: 1.5885 - accuracy: 0.6986 - val_loss: 0.6057 - val_accuracy: 0.7299
Epoch 3/20
846/846 [==============================] - 36s 43ms/step - loss: 9.4673 - accuracy: 0.7009 - val_loss: 0.6192 - val_accuracy: 0.7299
Epoch 4/20
846/846 [==============================] - 36s 43ms/step - loss: 19.0221 - accuracy: 0.7033 - val_loss: 0.6762 - val_accuracy: 0.6897
Epoch 5/20
846/846 [==============================] - 37s 43ms/step - loss: 12.9867 - accuracy: 0.7021 - val_loss: 0.8426 - val_accuracy: 0.7126
Epoch 6/20
846/846 [==============================] - 37s 44ms/step - loss: 4.5137 - accuracy: 0.7092 - val_loss: 1.0510 - val_accuracy: 0.7011
Epoch 7/20
846/846 [==============================] - 37s 43ms/step - loss: 2.1221 - accuracy: 0.7116 - val_loss: 0.6556 - val_acc

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


852/852 [==============================] - 38s 44ms/step - loss: 515.8306 - accuracy: 0.6620 - val_loss: 0.6301 - val_accuracy: 0.7798
Epoch 2/20
852/852 [==============================] - 37s 43ms/step - loss: 1.5359 - accuracy: 0.6878 - val_loss: 0.5500 - val_accuracy: 0.7857
Epoch 3/20
852/852 [==============================] - 36s 43ms/step - loss: 34.2718 - accuracy: 0.6960 - val_loss: 0.5206 - val_accuracy: 0.7857
Epoch 4/20
852/852 [==============================] - 36s 43ms/step - loss: 1.2874 - accuracy: 0.6913 - val_loss: 0.5817 - val_accuracy: 0.7857
Epoch 5/20
852/852 [==============================] - 37s 43ms/step - loss: 11.5605 - accuracy: 0.6878 - val_loss: 0.5189 - val_accuracy: 0.7857
Epoch 6/20
852/852 [==============================] - 37s 44ms/step - loss: 13.4791 - accuracy: 0.6901 - val_loss: 0.6366 - val_accuracy: 0.7857
Epoch 7/20
852/852 [==============================] - 37s 43ms/step - loss: 9.2488 - accuracy: 0.6866 - val_loss: 0.5635 - val_accuracy: 0.785

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


...

847/847 [==============================] - 38s 44ms/step - loss: 396.4515 - accuracy: 0.6600 - val_loss: 0.6386 - val_accuracy: 0.7110
Epoch 2/20
847/847 [==============================] - 37s 44ms/step - loss: 1.1206 - accuracy: 0.7072 - val_loss: 0.6195 - val_accuracy: 0.7110
Epoch 3/20
847/847 [==============================] - 37s 43ms/step - loss: 14.2060 - accuracy: 0.6848 - val_loss: 0.6053 - val_accuracy: 0.7110
Epoch 4/20
847/847 [==============================] - 36s 43ms/step - loss: 5.4565 - accuracy: 0.6989 - val_loss: 0.6134 - val_accuracy: 0.7110
Epoch 5/20
847/847 [==============================] - 37s 43ms/step - loss: 8.8344 - accuracy: 0.6860 - val_loss: 0.7340 - val_accuracy: 0.6185
Epoch 6/20
847/847 [==============================] - 37s 43ms/step - loss: 1.9954 - accuracy: 0.6895 - val_loss: 0.6509 - val_accuracy: 0.7052
Epoch 7/20
847/847 [==============================] - 37s 44ms/step - loss: 3.8080 - accuracy: 0.7025 - val_loss: 0.6537 - val_accuracy: 0.7110


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


858/858 [==============================] - 37s 42ms/step - loss: 286.8394 - accuracy: 0.6760 - val_loss: 0.6247 - val_accuracy: 0.6975
Epoch 2/20
858/858 [==============================] - 36s 42ms/step - loss: 29.6026 - accuracy: 0.6970 - val_loss: 0.6362 - val_accuracy: 0.6975
Epoch 3/20
858/858 [==============================] - 37s 43ms/step - loss: 1.5409 - accuracy: 0.7016 - val_loss: 14.7807 - val_accuracy: 0.6975
Epoch 4/20
858/858 [==============================] - 36s 42ms/step - loss: 1.1698 - accuracy: 0.7086 - val_loss: 0.7041 - val_accuracy: 0.6914
Epoch 5/20
858/858 [==============================] - 36s 42ms/step - loss: 16.1500 - accuracy: 0.6993 - val_loss: 0.9548 - val_accuracy: 0.6975
Epoch 6/20
858/858 [==============================] - 36s 42ms/step - loss: 12.2267 - accuracy: 0.7098 - val_loss: 0.9776 - val_accuracy: 0.6914
Epoch 7/20
858/858 [==============================] - 36s 42ms/step - loss: 4.6099 - accuracy: 0.7110 - val_loss: 0.6114 - val_accuracy: 0.70

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


860/860 [==============================] - 38s 44ms/step - loss: 520.1136 - accuracy: 0.6744 - val_loss: 0.6383 - val_accuracy: 0.6687
Epoch 2/20
860/860 [==============================] - 37s 43ms/step - loss: 3.8614 - accuracy: 0.7093 - val_loss: 0.6378 - val_accuracy: 0.6687
Epoch 3/20
860/860 [==============================] - 37s 42ms/step - loss: 8.8954 - accuracy: 0.7093 - val_loss: 0.6416 - val_accuracy: 0.6687
Epoch 4/20
860/860 [==============================] - 37s 43ms/step - loss: 11.7663 - accuracy: 0.7093 - val_loss: 0.6629 - val_accuracy: 0.6687
Epoch 5/20
860/860 [==============================] - 37s 43ms/step - loss: 7.5127 - accuracy: 0.7128 - val_loss: 0.7475 - val_accuracy: 0.6562
Epoch 6/20
860/860 [==============================] - 37s 43ms/step - loss: 3.0622 - accuracy: 0.7174 - val_loss: 0.6680 - val_accuracy: 0.6687
Epoch 7/20
860/860 [==============================] - 37s 43ms/step - loss: 18.8927 - accuracy: 0.7105 - val_loss: 0.8294 - val_accuracy: 0.6562

In [ ]:
def get_outputs(model,fold):
  outputs = []
  y_true = []
  model = tf.keras.models.load_model('drive/MyDrive/pretrained_fold'+str(model))
  for x in fold['validation']:
    y_true.append(labels[str(x)])
    img = Image.open('/content/drive/MyDrive/g1020-polygons/' + x[:-4] + '.png')
    background = Image.new("RGB", img.size, (255, 255, 255))
    background.paste(img, mask = img.split()[3])
    img = np.expand_dims(np.array(background.resize((299, 299), Image.ANTIALIAS)),axis = 0)
    outputs.append(np.argmax(model(img)))
    #print(outputs)
  return outputs,y_true

In [ ]:
for i in range(6):
  outputs,y_true = get_outputs(i,folds[i])
  print(classification_report(y_true,outputs))

              precision    recall  f1-score   support

           0       0.73      0.99      0.84       127
           1       0.50      0.02      0.04        47

    accuracy                           0.73       174
   macro avg       0.62      0.51      0.44       174
weighted avg       0.67      0.73      0.63       174

              precision    recall  f1-score   support

           0       0.79      0.92      0.85       132
           1       0.23      0.08      0.12        36

    accuracy                           0.74       168
   macro avg       0.51      0.50      0.49       168
weighted avg       0.67      0.74      0.69       168



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.67      1.00      0.80       122
           1       0.00      0.00      0.00        61

    accuracy                           0.67       183
   macro avg       0.33      0.50      0.40       183
weighted avg       0.44      0.67      0.53       183

              precision    recall  f1-score   support

           0       0.72      0.97      0.83       123
           1       0.50      0.08      0.14        50

    accuracy                           0.71       173
   macro avg       0.61      0.52      0.48       173
weighted avg       0.66      0.71      0.63       173

              precision    recall  f1-score   support

           0       0.25      0.01      0.02       113
           1       0.29      0.94      0.44        49

    accuracy                           0.29       162
   macro avg       0.27      0.47      0.23       162
weighted avg       0.26      0.29      0.15       162

              preci

# Knowledge Incorporated Model

In [ ]:
import numpy as np
import keras
import tensorflow
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=2, dim=(299,299), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        k = np.empty((self.batch_size,2))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = Image.open('/content/drive/MyDrive/g1020-polygons/' + ID[:-4] + '.png')
            background = Image.new("RGB", img.size, (255, 255, 255))
            background.paste(img, mask = img.split()[3])
            X[i,] = np.array(background.resize((299, 299), Image.ANTIALIAS))

            # Store class
            y[i] = self.labels[ID]
            k[i] = np.load('/content/drive/MyDrive/knowledge_g1020_best/'+str(ID[:-4])+'.npy',allow_pickle= True)

        return [X,k], tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
def knowledge_incorporated_model(pretrained_model):    
    input = Input((299,299,3))
    inputB = Input((2))
    inputB = Activation('softmax')(inputB)
    l1    = pretrained_model(input)
    l1    = Dense(2)(l1)
    l2    = Activation('sigmoid')(l1)
    l3    = Add()([l2,inputB])
    l4    = Dense(128)(l3)
    l5    = Activation('relu')(l4)
    l6    = Dense(2)(l5)
    l7    = Activation('sigmoid')(l6)
    l8    = Add()([l7,inputB])
    l9    = Activation('softmax')(l8)
    return Model(inputs=[input, inputB], outputs=l9)

In [ ]:
for i in range(6):
  training_generator_final = DataGenerator(folds[i]['train'], labels)
  validation_generator_final = DataGenerator(folds[i]['validation'], labels)
  pretrained_model = tf.keras.models.load_model('drive/MyDrive/pretrained_fold'+str(i))
  opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
  final_model = knowledge_incorporated_model(pretrained_model)
  final_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
  history_final = final_model.fit_generator(training_generator_final,validation_data=validation_generator_final,epochs = 20)
  tf.keras.models.save_model(final_model,'drive/MyDrive/G1020_knowledge_incorporated_fold'+str(i))

In [ ]:
for i in range(6):
  model = tf.keras.models.load_model('drive/MyDrive/G1020_knowledge_incorporated_fold'+str(i))
  y_true = []
  for x in folds[i]['validation']:
    y_true.append(labels[str(x)])
  validation_generator = DataGenerator(folds[i]['validation'], labels,batch_size=len(folds[i]['validation']))
  outputs = np.argmax(model.predict(validation_generator),axis = -1)
  print(outputs.shape)
  print(classification_report(y_true,outputs))


(174,)
              precision    recall  f1-score   support

           0       0.74      0.23      0.35       127
           1       0.27      0.79      0.41        47

    accuracy                           0.38       174
   macro avg       0.51      0.51      0.38       174
weighted avg       0.62      0.38      0.36       174

(168,)
              precision    recall  f1-score   support

           0       0.79      0.28      0.41       132
           1       0.21      0.72      0.33        36

    accuracy                           0.38       168
   macro avg       0.50      0.50      0.37       168
weighted avg       0.66      0.38      0.40       168

(183,)
              precision    recall  f1-score   support

           0       0.70      0.33      0.45       122
           1       0.35      0.72      0.47        61

    accuracy                           0.46       183
   macro avg       0.53      0.52      0.46       183
weighted avg       0.58      0.46      0.45       183

# Knowledge Distillation

In [ ]:
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Add, Reshape,Conv2D,MaxPooling2D,ZeroPadding2D,experimental,AveragePooling2D,Concatenate,Dropout,Activation
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import tensorflow
from PIL import Image

In [ ]:
def student_model():
    input = Input((299,299,3))
    c1l1  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(input)
    c1l2  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l1)
    ces2  = Add()([c1l2, c1l1])
    c1l3  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l2)
    ces3  = Add()([c1l3, ces2])
    ces3  = Flatten()(c1l3)
    ces3  = Dense(2)(ces3)
    ces3  = Activation('softmax')(ces3)
    return Model(inputs=input, outputs=ces3)

In [ ]:
import numpy as np
import keras
import tensorflow
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=16, dim=(299,299), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        k = np.empty((self.batch_size,2))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = Image.open('/content/drive/MyDrive/g1020-polygons/' + ID[:-4] + '.png')
            background = Image.new("RGB", img.size, (255, 255, 255))
            background.paste(img, mask = img.split()[3])
            X[i,] = np.array(background.resize((299, 299), Image.ANTIALIAS))

            # Store class
            y[i] = self.labels[ID]
            k[i] = np.load('/content/drive/MyDrive/knowledge_g1020_best/'+str(ID[:-4])+'.npy',allow_pickle= True)

        return [X,k], tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
for i in range(1,6):
  print('\n\nfold' + str(i) + '\n\n')  
  student = student_model()
  training_generator_final = DataGenerator(folds[i]['train'], labels)
  validation_generator_final = DataGenerator(folds[i]['validation'], labels)
  for epochs in range(20):
    for X,y in training_generator_final:
      x,k = X
      teacher_predictions = k
      student_loss_fn = tf.keras.losses.CategoricalCrossentropy()
      distillation_loss_fn = tf.keras.losses.KLDivergence()
      optimizer = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
      with tf.GradientTape() as tape:
          student_predictions = student(x, training=True)
          student_loss = student_loss_fn(y, student_predictions)
          distillation_loss = distillation_loss_fn(
              tf.nn.softmax(teacher_predictions / 3, axis=1),
              student_predictions
          )
          loss = 0.5 * student_loss + 0.5 * distillation_loss
      trainable_vars = student.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)
      optimizer.apply_gradients(zip(gradients, trainable_vars))
      metrics = [keras.metrics.SparseCategoricalAccuracy()]
      results = {m.name: m.result() for m in metrics}
      results.update({"student_loss": student_loss, "distillation_loss": distillation_loss})
      print('epoch: ' + str(epochs) + '/20')
      for name,value in results.items():
        print(name + ': ' + str(value)[10:18],end = ' ------ ')
      print('\n')
  tf.keras.models.save_model(student,'drive/MyDrive/G1020_student'+str(i))



fold1




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 46.88693 ------ distillation_loss: 2.758410 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 166553.6 ------ distillation_loss: 11.32229 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 6020.168 ------ distillation_loss: 10.26587 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 150055.3 ------ distillation_loss: 2.328705 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 200.1944 ------ distillation_loss: 12.76599 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 30729.56 ------ distillation_loss: 4.525665 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 15269.72 ------ distillation_loss: 10.44933 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 3359.896 ------ distillation_loss: 1.545037 ------ 

epoch: 0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 13.26019 ------ distillation_loss: 10.09697 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 107809.0 ------ distillation_loss: 2.819065 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 3441.970 ------ distillation_loss: 13.42853 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 30385.24 ------ distillation_loss: 4.84441, ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 860.595, ------ distillation_loss: 10.15111 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 1594.017 ------ distillation_loss: 4.249346 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 68.54922 ------ distillation_loss: 13.58157 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 379.4178 ------ distillation_loss: 3.864315 ------ 

epoch: 0

...

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 67.49866 ------ distillation_loss: 2.224200 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 160950.6 ------ distillation_loss: 11.10521 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 7567.497 ------ distillation_loss: 4.935228 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 40071.61 ------ distillation_loss: 8.754084 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 206.4495 ------ distillation_loss: 5.902947 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 15104.08 ------ distillation_loss: 11.15333 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 38795.74 ------ distillation_loss: 2.431914 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 509.9778 ------ distillation_loss: 11.74213 ------ 

epoch: 0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 3.515829 ------ distillation_loss: 6.18524, ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 216580.6 ------ distillation_loss: 5.403401 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 3993.056 ------ distillation_loss: 12.42775 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 57528.43 ------ distillation_loss: 3.811560 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 413.2216 ------ distillation_loss: 12.18862 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 3787.716 ------ distillation_loss: 5.085125 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 3548.284 ------ distillation_loss: 12.65831 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 5015.205 ------ distillation_loss: 5.09153, ------ 

epoch: 0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 2.181808 ------ distillation_loss: 1.678681 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 142578.1 ------ distillation_loss: 3.711253 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 5790.979 ------ distillation_loss: 11.67349 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 40466.9, ------ distillation_loss: 3.288405 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 483.4254 ------ distillation_loss: 12.78182 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 30246.57 ------ distillation_loss: 4.135454 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 1959.659 ------ distillation_loss: 11.77211 ------ 

epoch: 0/20
sparse_categorical_accuracy: 0.0, sha ------ student_loss: 13.00921 ------ distillation_loss: 3.587884 ------ 

epoch: 0

In [ ]:
for i in range(6):
  y_true = []
  y_pred = []
  val_gen = DataGenerator(folds[i]['validation'], labels)
  model = tf.keras.models.load_model('/content/drive/MyDrive/G1020_student'+str(i))
  for x,y in val_gen:
    y_pred.append(np.argmax(model(x)))
    y_true.append(y)
  print(classification_report(y_true,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.73      1.00      0.84       127
           1       0.00      0.00      0.00        47

    accuracy                           0.73       174
   macro avg       0.36      0.50      0.42       174
weighted avg       0.53      0.73      0.62       174



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       132
           1       0.21      1.00      0.35        36

    accuracy                           0.21       168
   macro avg       0.11      0.50      0.18       168
weighted avg       0.05      0.21      0.08       168

              precision    recall  f1-score   support

           0       0.63      0.43      0.51       122
           1       0.30      0.49      0.37        61

    accuracy                           0.45       183
   macro avg       0.46      0.46      0.44       183
weighted avg       0.52      0.45      0.46       183



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       123
           1       0.29      1.00      0.45        50

    accuracy                           0.29       173
   macro avg       0.14      0.50      0.22       173
weighted avg       0.08      0.29      0.13       173

              precision    recall  f1-score   support

           0       1.00      0.03      0.05       113
           1       0.31      1.00      0.47        49

    accuracy                           0.32       162
   macro avg       0.65      0.51      0.26       162
weighted avg       0.79      0.32      0.18       162

              precision    recall  f1-score   support

           0       0.67      1.00      0.80       107
           1       0.00      0.00      0.00        53

    accuracy                           0.67       160
   macro avg       0.33      0.50      0.40       160
weighted avg       0.45      0.67      0.54       160



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
